# 1. SETTINGS

In [116]:
# libraries
import numpy as np
import pandas as pd
from datetime import date

In [117]:
# warnings
import warnings
warnings.filterwarnings("ignore")

In [118]:
# plots
import matplotlib as plt
%matplotlib inline

In [119]:
# pandas options
pd.set_option("display.max_columns", None)

# 2. IMPORT

In [120]:
# import datasets
data = pd.read_csv("../data/prepared/data_basic_1.csv", compression = "gzip")

In [121]:
# check data
print("Dimensions:", data.shape)
data.head()

Dimensions: (111778638, 6)


,PredictionIdx,CustomerIdx,IsinIdx,BuySell,CustomerInterest,Week
0,NaN,0,16471,Buy,0.0,1
1,NaN,0,16471,Buy,0.0,2
2,NaN,0,16471,Buy,0.0,3
3,NaN,0,16471,Buy,0.0,4
4,NaN,0,16471,Buy,0.0,5


# 3. FUNCTIONS TO CREATE FEATURES

In [122]:
### FUNCTION FOR COMPUTING WEEK INDEX
def week_idx(date, end_date):
    return round((end_date - date).dt.days / 7 + 0.4).astype(int)

### RECENCY (TIME SINCE LAST TRADE)

In [123]:
##### FUNCTION TO COMPUTE 6 RECENCY-BASED FEATURES 
# 1) Time since last customer trade with that bond with the same BuySell direction
# 2) Time since last customer trade with that bond with any direction
# 3) Time since last customer trade with any bond with the same BuySell direction
# 4) Time since last customer trade with any bond with any direction
# 5) Time since last trade with that bond by any of the customers with the same BuySell direction
# 6) Time since last trade with that bond by any of the customers

def compute_recency(data):

    tmp = data[data.CustomerInterest == 1]
    
    tmp["PrevWeek"] = tmp.sort_values(by = ["Week"], ascending = True).groupby(["CustomerIdx", "IsinIdx", "BuySell"]).Week.shift(1)
    tmp["CurrWeek"] = tmp.Week

    data = data.merge(tmp, how = "left")

    data["CurrWeek"] = data.groupby(["CustomerIdx", "IsinIdx", "BuySell"]).CurrWeek.fillna(method = "ffill")
    data["PrevWeek"] = data.groupby(["CustomerIdx", "IsinIdx", "BuySell"]).PrevWeek.fillna(method = "bfill")
    data["PrevWeek"][data.PrevWeek.isnull()] = data["CurrWeek"]
    data["PrevWeek"][data.PrevWeek >= data.Week] = None
    
    data["Recency1"] = data["Week"] - data["PrevWeek"]
    data["Recency1"][data.Recency1 <= 0] = None
    del data["PrevWeek"], data["CurrWeek"]

    Recency2 = data.groupby(["CustomerIdx", "IsinIdx", "Week"]).Recency1.min().reset_index()
    Recency2.columns = ["CustomerIdx", "IsinIdx", "Week", "Recency2"]
    data = data.merge(Recency2, how = "left", on = ["CustomerIdx", "IsinIdx", "Week"])
    
    Recency3 = data.groupby(["CustomerIdx", "BuySell", "Week"]).Recency1.min().reset_index()
    Recency3.columns = ["CustomerIdx", "BuySell", "Week", "Recency3"]
    data = data.merge(Recency3, how = "left", on = ["CustomerIdx", "BuySell", "Week"])
    
    Recency4 = data.groupby(["CustomerIdx", "Week"]).Recency1.min().reset_index()
    Recency4.columns = ["CustomerIdx", "Week", "Recency4"]
    data = data.merge(Recency4, how = "left", on = ["CustomerIdx", "Week"])
    
    Recency5 = data.groupby(["IsinIdx", "BuySell", "Week"]).Recency1.min().reset_index()
    Recency5.columns = ["IsinIdx", "BuySell", "Week", "Recency5"]
    data = data.merge(Recency5, how = "left", on = ["IsinIdx", "BuySell", "Week"])
    
    Recency6 = data.groupby(["IsinIdx", "Week"]).Recency1.min().reset_index()
    Recency6.columns = ["IsinIdx", "Week", "Recency6"]
    data = data.merge(Recency6, how = "left", on = ["IsinIdx", "Week"])
    
    data.Recency1.fillna(data.Recency1.max(), inplace = True)
    data.Recency2.fillna(data.Recency2.max(), inplace = True)
    data.Recency3.fillna(data.Recency3.max(), inplace = True)
    data.Recency4.fillna(data.Recency4.max(), inplace = True)
    data.Recency5.fillna(data.Recency5.max(), inplace = True)
    data.Recency6.fillna(data.Recency6.max(), inplace = True)
    
    print("Computed 6 recency features...")
    
    return(data)

### FREQUENCY (TOTAL NUMBER OF TRADES)

In [124]:
##### FUNCTION TO COMPUTE 6 FREQUENCY-BASED FEATURES 
# 1) Number of previous customer trades with that bond with the same BuySell direction
# 2) Number of previous customer trades with that bond with any direction
# 3) Number of previous customer trades with any bond with the same BuySell direction
# 4) Number of previous customer trades with any bond with any direction
# 5) Number of previous trades with that bond by any of the same BuySell direction
# 6) Number of previous trades with that bond by any of the customers
# all frequencies are divided by the number of weeks during which they were observed

def compute_frequency(data):

    tmp = data[data.CustomerInterest == 1]

    tmp["Frequency1"] = tmp.sort_values(by = ["Week"], ascending = True).groupby(["CustomerIdx", "IsinIdx", "BuySell"]).CustomerInterest.cumsum()

    data = data.merge(tmp, how = "left")
    data["Frequency1"] = data.groupby(["CustomerIdx", "IsinIdx", "BuySell"]).Frequency1.fillna(method = "ffill")
    data["Frequency1"][data["CustomerInterest"] == 1] = data["Frequency1"][data["CustomerInterest"] == 1] - 1
    data["Frequency1"].fillna(0, inplace = True)

    Frequency2 = data.groupby(["CustomerIdx", "IsinIdx", "Week"]).Frequency1.sum().reset_index()
    Frequency2.columns = ["CustomerIdx", "IsinIdx", "Week", "Frequency2"]
    data = data.merge(Frequency2, how = "left", on = ["CustomerIdx", "IsinIdx", "Week"])
    
    Frequency3 = data.groupby(["CustomerIdx", "BuySell", "Week"]).Frequency1.sum().reset_index()
    Frequency3.columns = ["CustomerIdx", "BuySell", "Week", "Frequency3"]
    data = data.merge(Frequency3, how = "left", on = ["CustomerIdx", "BuySell", "Week"])

    Frequency4 = data.groupby(["CustomerIdx", "Week"]).Frequency1.sum().reset_index()
    Frequency4.columns = ["CustomerIdx", "Week", "Frequency4"]
    data = data.merge(Frequency4, how = "left", on = ["CustomerIdx", "Week"])

    Frequency5 = data.groupby(["IsinIdx", "Week", "BuySell"]).Frequency1.sum().reset_index()
    Frequency5.columns = ["IsinIdx", "Week", "BuySell", "Frequency5"]
    data = data.merge(Frequency5, how = "left", on = ["IsinIdx", "Week", "BuySell"])
    
    Frequency6 = data.groupby(["IsinIdx", "Week"]).Frequency1.sum().reset_index()
    Frequency6.columns = ["IsinIdx", "Week", "Frequency6"]
    data = data.merge(Frequency6, how = "left", on = ["IsinIdx", "Week"])
    
    div = data.Week - data.Week.min()
    div[div == 0] = 1
    for var in ["Frequency1", "Frequency2", "Frequency3", "Frequency4", "Frequency5", "Frequency6"]:
        data[var] = data[var] / div
    
    print("Computed 6 frequency features...")

    return(data)

### MONTH ID

In [125]:
##### FUNCTION TO COMPUTE MONTH
# 1) Month of each trade

def compute_month(data):

    tmp = pd.read_csv("../data/raw/Trade.csv")
    tmp = tmp[["TradeDateKey"]]

    ### FUNCTION FOR COMPUTING WEEK INDEX
    def week_idx(date, end_date):
        return round((end_date - date).dt.days / 7 + 0.4).astype(int)

    tmp["TradeDateKey"] = pd.to_datetime(tmp["TradeDateKey"], format = '%Y%m%d')
    tmp["Week"] = week_idx(tmp["TradeDateKey"], pd.Timestamp('2018-04-23 00:00:00'))
    tmp["Month"] = tmp["TradeDateKey"].dt.month.astype("object")
    del tmp["TradeDateKey"]
    tmp = tmp.drop_duplicates()

    data = data.merge(tmp, how = "left", on = "Week")
    data["Month"][data.Week == 121] = "4"
    
    print("Computed 1 month feature...")
    
    return data

### INDICATORS OF CUSTOMER INFORMATION

In [126]:
cust  = pd.read_csv("../data/raw/Customer.csv")

In [127]:
cust.loc[cust.Subsector=="Bank","Sector"] = "Bank"
cust.loc[cust.Subsector=="Broker Dealer","Sector"] = "Broker"
cust.loc[cust.Subsector=="Hedge Fund","Sector"] = "Hedgefund"
cust.loc[cust.Subsector=="Independent Asset Manager","Sector"] = "Independent"

In [128]:
# create dummies for customers
cust_dummies = pd.concat([cust.CustomerIdx, pd.get_dummies(cust.Sector), pd.get_dummies(cust.Region)], axis = 1)

### BOND-SPECIFIC FINANCIAL INDICATORS

In [129]:
bond  = pd.read_csv("../data/raw/Isin.csv")

In [130]:
bond.CouponType.unique()

array(['STEP CPN', 'FIXED', 'FLOATING', 'ZERO COUPON', 'VARIABLE', 'NONE'], dtype=object)

In [131]:
# create bond dummies
bond_dummies = pd.concat([bond.IsinIdx,
                          pd.get_dummies(bond.ActivityGroup), 
                          pd.get_dummies(bond.CompositeRating),
                          pd.get_dummies(bond.CouponType)
                         ], axis = 1)

### MACRO VARIABLES

In [132]:
macro = pd.read_csv("../data/raw/MarketData_Macro.csv")

We have >100 variables here:
- Stock indices (DAX, FTSE100, ...)
- Volatility indices (VSTOXX, VIX, )
- Currency exchange rates (USD <> EUR/CNY/...)
- Inter-bank money lending rate (Money Market) 3-month for each currency 
- Mid- to long-term swaps (2-10 years). TODO: Unsure of the effect on bond trades

In [133]:
# descriptive stats
macro.columns.tolist()[0:15]

['DateKey',
 'SSE',
 'DAX',
 'EUROSTOXX',
 'VSTOXX',
 'FTSE100',
 'HSI',
 'NIKKEI',
 'DOWJONES_INDU',
 'SP500',
 'VIX',
 'FX_USD.ARS',
 'FX_USD.AUD',
 'FX_USD.BRL',
 'FX_USD.CAD']

Heuristically fill missing values with the previous value or 2xprevious value. If still missing, fill values with the following or 2xfollowing value

In [134]:
macro = macro.fillna(macro.shift(1)).fillna(macro.shift(2)).fillna(macro.shift(-1)).fillna(macro.shift(-2))

In [135]:
# convert dates
macro["DateKey"] = pd.to_datetime(macro["DateKey"], format = '%Y%m%d')

In [136]:
### FUNCTION FOR COMPUTING WEEK INDEX
def week_idx(date, end_date):
    return round((end_date - date).dt.days / 7 + 0.4).astype(int)

# add week index
macro["Week"] = week_idx(macro["DateKey"], pd.Timestamp('2018-04-23 00:00:00'))
macro["Week"] = macro["Week"].max() + 1 - macro["Week"]

Aggregate the macro values by week. 

TODO: We could also take the lag first and then aggregate, not sure what makes more sense (JH).

In [137]:
macro = macro.groupby(["Week"]).agg("mean")

We are interested in the change in the macro variable compared to the previous date, I think, to check if e.g. the currency value went up or down

In [138]:
# Replace missing lag for first week with 0
macro_diff1 = ((macro - macro.shift(1))/macro).fillna(0)
#macro_diff1.columns = [x + "_diff1" for x in macro_diff1.columns]

In [139]:
indices_trend = macro_diff1\
    .filter(["SSE","DAX","EUROSTOXX","VSTOXX","FTSE100","HSI","NIKKEI","DOWJONAES_INDU","SP500","VIX"])\
    .reset_index()

TODO: I think it makes sense to create a common variable e.g. "currency trend" that relates to the specific currency of the bond and/or holder.

In [140]:
fx_diff1 = macro_diff1.filter(like = "FX", axis = 1)

In [141]:
fx_diff1.columns

Index(['FX_USD.ARS', 'FX_USD.AUD', 'FX_USD.BRL', 'FX_USD.CAD', 'FX_USD.CHF',
       'FX_USD.CNO', 'FX_USD.CNY', 'FX_USD.EUR', 'FX_USD.GBP', 'FX_USD.HKD',
       'FX_USD.IDR', 'FX_USD.JPY', 'FX_USD.NOK', 'FX_USD.SGD', 'FX_USD.TRY',
       'FX_USD.ZAR'],
      dtype='object')

In [142]:
fx_diff1["USD"] = 1
#fx_diff1["USD_diff1"] = 1

In [143]:
fx_diff1 = fx_diff1.reset_index().melt(id_vars = "Week", var_name = "Currency", value_name = "Currency_trend")

In [ ]:
fx_diff1.Currency = fx_diff1.Currency.str[-3:]

### BOND DATE-RELATED FEATURES

In [ ]:
# load dataset
bond  = pd.read_csv("../data/raw/Isin.csv")
bond.head()

In [ ]:
### CONVERT DATES TO WEEKS RELATIVE TO THE SAME BASE

def week_idx(date, end_date):
    return round((end_date - date).dt.days / 7 + 0.4).astype(int)

# convert dates
bond["ActualMaturityDateKey"] = pd.to_datetime(bond["ActualMaturityDateKey"], format = '%Y%m%d')
bond["IssueDateKey"]          = pd.to_datetime(bond["IssueDateKey"], format = '%Y%m%d')

# add week index
bond["ActualMaturityDateKey"] = week_idx(bond["ActualMaturityDateKey"], pd.Timestamp('2018-04-23 00:00:00'))
bond["IssueDateKey"] = week_idx(bond["IssueDateKey"], pd.Timestamp('2018-04-23 00:00:00'))

# subset
bond = bond[["IsinIdx", "ActualMaturityDateKey", "IssueDateKey"]]

### MARKET DYNAMIC FEATURES

In [ ]:
# load dataset
market  = pd.read_csv("../data/raw/Market.csv")

# convert dates
market["DateKey"] = pd.to_datetime(market["DateKey"], format = '%Y%m%d')

# add week index
market["Week"] = week_idx(market["DateKey"], pd.Timestamp('2018-04-23 00:00:00'))
del market["DateKey"]

In [ ]:
# compute weekly difference
tmp = market.groupby(["IsinIdx", "Week"]).agg(["mean"])
tmp.columns = ["_lastweekdiff_".join(col).strip() for col in tmp.columns.values]
tmp = tmp.groupby("IsinIdx").diff()
tmp = tmp.reset_index()

In [ ]:
# compute weekly SD
tmp2 = market.groupby(["IsinIdx", "Week"]).agg(["std"])
tmp2.columns = ["_lastweek_".join(col).strip() for col in tmp2.columns.values]
tmp2 = tmp2.reset_index()
tmp2.Week = tmp2.Week + 1

In [ ]:
# merge data
market = tmp.merge(tmp2, how = "left")

# 4. COMPUTING FEATURES

### MERGE FEATURES FROM NIKITA

In [ ]:
# compute past features
data = compute_recency(data)
data = compute_frequency(data)

In [ ]:
# compute months
data = compute_month(data)

In [ ]:
# compute BuySell dummy
data["Buy"] = 0
data["Buy"][data.BuySell == "Buy"] = 1 

In [ ]:
### BOND AND MARKET FEATURES

# merge data
data = data.merge(bond, on = "IsinIdx", how = "left")
data = data.merge(market, on = ["IsinIdx", "Week"], how = "left")

# compute day differences
data["ActualMaturityDateKey"] = data["ActualMaturityDateKey"] - data["Week"]
data["IssueDateKey"]          = data["IssueDateKey"] - data["Week"]

### MERGE FEATURES FROM JOHANNES

In [ ]:
# Customer Purchase distribution moments
data = data.merge(
    data.groupby(["CustomerIdx","Week"]).CustomerInterest.sum().groupby("CustomerIdx").std().reset_index()\
        .filter(["CustomerIdx","CustomerInterest"], axis=1)\
        .rename(columns={'CustomerInterest':'week_trading_std'}),
    how = "left", on = ["CustomerIdx"])

In [ ]:
# merge customer dummies
data = data.merge(cust_dummies, on = "CustomerIdx", how = "left")

In [ ]:
# merge bond dummies
bond_dummies = bond_dummies[['IsinIdx', 'FLOW G10', 'FLOW LOCAL MARKET', 'SAS & COVERED BONDS', 'NR']]
data = data.merge(bond_dummies, on = "IsinIdx", how = "left")

In [ ]:
# merge macro variables
data = data.merge(bond[["IsinIdx", "Currency"]], on = "IsinIdx")
data = data.merge(fx_diff1, how = 'left', on = ["Week","Currency"])
data = data.merge(indices_trend, how = 'left', on = ["Week"])

# there are a few weird currencies (or typos?) for which we don't have information, e.g. CNH. These need to be imputed
data[fx_diff1.columns] = data[fx_diff1.columns].fillna(0)
data.drop("Currency", axis = 1, inplace = True)

In [ ]:
# merge exchange rates
data = data.merge(macro_diff1.reset_index(), on = "Week", how = "left")

### MERGE FEATURES FROM ALISA

In [ ]:
### ADD NEW BOND FEATURES [V1]

# load bond data
bond_stat = pd.read_csv("../data/prepared/bondstat1_AK")

# merge bond data
data = data.merge(bond_stat[["IsinIdx", "MeanPrice", "StdPrice", "MeanYield",
      "StdYield", "MeanZScore", "StdZScore", "YieldMarktDelta",
      "ZScoreMarktDelta"]], how = "left", on = "IsinIdx")

In [ ]:
### ADD NEW BOND FEATURES [V2]

# load bond data
bond_stat = pd.read_csv("../data/prepared/bondstat2_AK")

# merge bond data
data = data.merge(bond_stat[["IsinIdx", 'ymeansq', 'ymeancube', 'zmeansq', 'zmeancube',
                             'years_tomaturity', 'count', 'ratingspread', 'ratingZspread']], 
                  how = "left", on = "IsinIdx")
print(data.shape)

In [ ]:
### ADD BOND FEATURES [V3]

# load bond data
bond_stat = pd.read_csv("../data/prepared/bondstat3_AK")
bond_stat.head()

# merge bond data
data = data.merge(bond_stat[["IsinIdx", 'predyield', 'predprice']], how = "left", on = "IsinIdx")
print(data.shape)

# 5. CORRECT AND EXPORT

In [ ]:
# clear memory
del bond_stat, macro_diff1, fx_diff1, bond, bond_dummies, cust_dummies

In [ ]:
# check recency distribution
data.Recency1.hist()

In [ ]:
# remove data with too high Recency2
data = data.loc[data.Recency2 < 27]

In [80]:
# remove first X weeks
data = data[data.Week > 50]

In [81]:
# check dimensions
print(data.shape)

(74680678, 19)


In [83]:
# check NA
nas = data.isnull().sum()
nas[nas > 0]

PredictionIdx       26650982
CustomerInterest      484758
Recency1             1868028
Recency3                4114
Recency5               16828
dtype: int64

In [85]:
# export CSV
data.to_csv("../data/prepared/data_v8_50_basic.csv", index = False, compression = "gzip")